In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

In [2]:
data_dir = 'data/'

image_size = 299
batch_size = 16

In [3]:
def get_class_weight(ds):
    y = np.concatenate([y for x, y in ds], axis=0)
    class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(y),
                                                 y = y)
    return class_weights

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(data_dir+"train", seed=123, 
                                                       image_size=(image_size, image_size), 
                                                       batch_size=batch_size)
class_names = train_ds.class_names
num_classes = len(class_names)

class_weights = get_class_weight(train_ds)
class_weights = dict(enumerate(class_weights))

Found 8012 files belonging to 7 classes.
Metal device set to: Apple M2


2023-01-27 18:36:25.542292: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-27 18:36:25.542394: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-01-27 18:36:25.597604: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(data_dir+'val',
                                                     image_size=(image_size, image_size), 
                                                     batch_size=32,  
                                                     shuffle=False, 
                                                     labels=None)

Found 1002 files belonging to 1 classes.


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [7]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.inception_v3.preprocess_input

In [8]:
base_model = tf.keras.applications.inception_v3.InceptionV3(input_shape=(image_size, image_size, 3), 
                                                            include_top=False, weights='imagenet')

In [9]:
train_ds = train_ds.map(lambda x,y : (preprocess_input(data_augmentation(x, training=True)), y))
val_ds = val_ds.map(lambda x: preprocess_input(x))

In [10]:
train_base = True
base_model.trainable = train_base

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
classification_layer = tf.keras.layers.Dense(1024, activation='relu')
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

x = base_model.layers[-1].output

x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = classification_layer(x)
outputs = prediction_layer(x)

model = tf.keras.Model(base_model.input, outputs)

In [11]:
base_learning_rate = 1e-5

loss_fn   = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=["acc"])

In [12]:
epochs = 30

In [13]:
def evaluate(model, ds, folder):
    df = pd.read_csv(f'data/{folder}_truth.csv')[['image_id','dx']]
    df = df.sort_values(by = 'image_id')
    class_names = sorted(list(set(df['dx'])))
    class_names = {name : idx for idx, name in enumerate(class_names)}
    df['dx'] = df['dx'].apply(lambda x: class_names[x])
    y_actual = list(df['dx'])
        
    y_pred = tf.convert_to_tensor([])
    for batch_img in ds:
        logit = model(batch_img, training = False)
        pred = np.argmax(logit, axis = 1)
        y_pred = tf.concat([y_pred, pred], axis = 0)

    y_pred = y_pred.numpy()
    print("acc :", accuracy_score(y_actual, y_pred))

In [14]:
_ = model.fit(train_ds, epochs = epochs, class_weight = class_weights)

InvalidArgumentError: Cannot assign a device for operation sequential/random_rotation/stateful_uniform/RngReadAndSkip: Could not satisfy explicit device specification '' because the node {{colocation_node sequential/random_rotation/stateful_uniform/RngReadAndSkip}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
RngReadAndSkip: CPU 
_Arg: GPU CPU 

Colocation members, user-requested devices, and framework assigned device

In [ ]:
evaluate(model, val_ds, 'val')

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(data_dir+'test',
                                                     image_size=(image_size, image_size), 
                                                     batch_size=32, 
                                                     shuffle=False, 
                                                     labels=None)
test_ds = test_ds.map(lambda x: preprocess_input(x))   
evaluate(model, test_ds, 'test')

In [ ]:
model.save('model/inception')